In [1]:
from replenishment import ArticleSimulationConfig, ReorderPointPolicy, simulate_replenishment_for_articles

base_policy = ReorderPointPolicy(reorder_point=20, order_quantity=50)
articles = {
    "article-1001": ArticleSimulationConfig(
        periods=6,
        demand=[15, 10, 25, 30, 5, 12],
        initial_on_hand=40,
        lead_time=2,
        policy=base_policy,
        holding_cost_per_unit=0.8,
        stockout_cost_per_unit=3.5,
        order_cost_per_order=9.5,
    ),
    "article-1002": ArticleSimulationConfig(
        periods=6,
        demand=[9, 14, 18, 12, 8, 11],
        initial_on_hand=30,
        lead_time=1,
        policy=base_policy,
        holding_cost_per_unit=0.8,
        stockout_cost_per_unit=3.5,
        order_cost_per_order=9.5,
    ),
}

results = simulate_replenishment_for_articles(articles)
print({article_id: result.summary for article_id, result in results.items()})


{'article-1001': SimulationSummary(total_demand=97, total_fulfilled=87, total_backorders=10, fill_rate=0.8969072164948454, average_on_hand=16.333333333333332, holding_cost=78.4, stockout_cost=35.0, total_cost=113.4), 'article-1002': SimulationSummary(total_demand=72, total_fulfilled=72, total_backorders=0, fill_rate=1.0, average_on_hand=28.5, holding_cost=136.8, stockout_cost=0.0, total_cost=136.8)}


In [ ]:
from replenishment import simulate_replenishment_with_aggregation, optimize_aggregation_windows

# Order-cycle (time aggregation) example: hard-code a cadence or optimize over it.
aggregation_window = 2
article = articles["article-1001"]
aggregated_result = simulate_replenishment_with_aggregation(
    periods=article.periods,
    demand=article.demand,
    initial_on_hand=article.initial_on_hand,
    lead_time=article.lead_time,
    policy=article.policy,
    aggregation_window=aggregation_window,
    holding_cost_per_unit=article.holding_cost_per_unit,
    stockout_cost_per_unit=article.stockout_cost_per_unit,
    order_cost_per_order=article.order_cost_per_order,
)
aggregated_result.summary

candidate_windows = [1, 2, 3]
optimized_windows = optimize_aggregation_windows(articles, candidate_windows)
print({article_id: result.window for article_id, result in optimized_windows.items()})


In [2]:
from replenishment import ArticleSimulationConfig, ForecastBasedPolicy, simulate_replenishment_for_articles

forecast_articles = {
    "article-2001": ArticleSimulationConfig(
        periods=6,
        demand=[20, 21, 19, 18, 22, 24],
        initial_on_hand=35,
        lead_time=1,
        policy=ForecastBasedPolicy(
            forecast=[18, 22, 20, 19, 21, 23],
            actuals=[20, 21, 19, 18, 22, 24],
            lead_time=1,
            service_level_factor=1.2,
        ),
        holding_cost_per_unit=0.8,
        stockout_cost_per_unit=3.5,
        order_cost_per_order=9.5,
    ),
    "article-2002": ArticleSimulationConfig(
        periods=6,
        demand=[14, 16, 13, 15, 17, 18],
        initial_on_hand=28,
        lead_time=1,
        policy=ForecastBasedPolicy(
            forecast=[15, 17, 16, 14, 18, 19],
            actuals=[14, 16, 13, 15, 17, 18],
            lead_time=1,
            service_level_factor=1.0,
        ),
        holding_cost_per_unit=0.8,
        stockout_cost_per_unit=3.5,
        order_cost_per_order=9.5,
    ),
}

forecast_results = simulate_replenishment_for_articles(forecast_articles)
print({article_id: result.summary for article_id, result in forecast_results.items()})


{'article-2001': SimulationSummary(total_demand=124, total_fulfilled=124, total_backorders=0, fill_rate=1.0, average_on_hand=20.5, holding_cost=98.4, stockout_cost=0.0, total_cost=98.4), 'article-2002': SimulationSummary(total_demand=93, total_fulfilled=93, total_backorders=0, fill_rate=1.0, average_on_hand=18.166666666666668, holding_cost=87.2, stockout_cost=0.0, total_cost=87.2)}


In [3]:
from replenishment import optimize_service_level_factors, optimize_aggregation_and_service_level_factors

# Service-level optimization (safety stock factor).
candidate_factors = [0.5, 1.0, 1.5, 2.0]
optimized = optimize_service_level_factors(forecast_articles, candidate_factors)
print({article_id: result.service_level_factor for article_id, result in optimized.items()})

# Joint optimization: aggregation window + service level factor.
candidate_windows = [1, 2, 3]
optimized_joint = optimize_aggregation_and_service_level_factors(
    forecast_articles,
    candidate_windows=candidate_windows,
    candidate_factors=candidate_factors,
)
print({article_id: (result.window, result.service_level_factor) for article_id, result in optimized_joint.items()})


{'article-2001': 0.5, 'article-2002': 0.5}


In [ ]:
from replenishment import ArticleSimulationConfig, PointForecastOptimizationPolicy, PercentileForecastOptimizationPolicy, simulate_replenishment_for_articles

safety_stock_articles = {
    "article-2101": ArticleSimulationConfig(
        periods=6,
        demand=[16, 18, 17, 20, 19, 21],
        initial_on_hand=32,
        lead_time=1,
        policy=PointForecastOptimizationPolicy(
            forecast=[15, 17, 18, 19, 20, 20],
            actuals=[16, 18, 17, 20, 19, 21],
            lead_time=1,
            service_level_factor=1.4,
        ),
        holding_cost_per_unit=0.7,
        stockout_cost_per_unit=3.2,
        order_cost_per_order=9.5,
    ),
}

percentile_articles = {
    "article-2201": ArticleSimulationConfig(
        periods=6,
        demand=[11, 13, 12, 14, 15, 13],
        initial_on_hand=24,
        lead_time=2,
        policy=PercentileForecastOptimizationPolicy(
            forecast=[12, 14, 13, 15, 16, 14],
            lead_time=2,
        ),
        holding_cost_per_unit=0.6,
        stockout_cost_per_unit=3.8,
        order_cost_per_order=9.5,
    ),
}

safety_stock_results = simulate_replenishment_for_articles(safety_stock_articles)
percentile_results = simulate_replenishment_for_articles(percentile_articles)
print({article_id: result.summary for article_id, result in safety_stock_results.items()})
print({article_id: result.summary for article_id, result in percentile_results.items()})


In [4]:
from replenishment import ForecastCandidatesConfig, optimize_forecast_targets, optimize_aggregation_and_forecast_targets, simulate_replenishment

# Percentile/forecast-target optimization.
forecast_candidates = {
    'article-3001': {
        'mean': [19, 20, 19, 20, 21, 21],
        '45-low': [18, 19, 18, 19, 20, 20],
        '45-high': [22, 23, 22, 23, 24, 24],
    },
    'article-3002': {
        'mean': [13, 14, 13, 14, 15, 15],
        '45-low': [12, 13, 12, 13, 14, 14],
        '45-high': [16, 17, 16, 17, 18, 18],
    },
}

candidate_configs = {
    'article-3001': ForecastCandidatesConfig(
        periods=6,
        demand=[18, 20, 19, 21, 22, 20],
        initial_on_hand=30,
        lead_time=1,
        forecast_candidates=forecast_candidates['article-3001'],
        holding_cost_per_unit=0.8,
        stockout_cost_per_unit=3.5,
        order_cost_per_order=9.5,
    ),
    'article-3002': ForecastCandidatesConfig(
        periods=6,
        demand=[12, 14, 13, 15, 16, 14],
        initial_on_hand=20,
        lead_time=2,
        forecast_candidates=forecast_candidates['article-3002'],
        holding_cost_per_unit=0.6,
        stockout_cost_per_unit=4.0,
        order_cost_per_order=9.5,
    ),
}

optimized_targets = optimize_forecast_targets(candidate_configs)
print({article_id: result.target for article_id, result in optimized_targets.items()})

final_results = {
    article_id: simulate_replenishment(
        periods=config.periods,
        demand=config.demand,
        initial_on_hand=config.initial_on_hand,
        lead_time=config.lead_time,
        policy=optimized_targets[article_id].policy,
        holding_cost_per_unit=config.holding_cost_per_unit,
        stockout_cost_per_unit=config.stockout_cost_per_unit,
        order_cost_per_order=config.order_cost_per_order,
    )
    for article_id, config in candidate_configs.items()
}
print({article_id: result.summary for article_id, result in final_results.items()})

# Joint optimization: aggregation window + forecast target.
candidate_windows = [1, 2, 3]
optimized_joint_targets = optimize_aggregation_and_forecast_targets(
    candidate_configs,
    candidate_windows=candidate_windows,
)
print({article_id: (result.window, result.target) for article_id, result in optimized_joint_targets.items()})


{'article-3001': '45-low', 'article-3002': '45-high'}
{'article-3001': SimulationSummary(total_demand=120, total_fulfilled=120, total_backorders=0, fill_rate=1.0, average_on_hand=8.833333333333334, holding_cost=42.400000000000006, stockout_cost=0.0, total_cost=42.400000000000006), 'article-3002': SimulationSummary(total_demand=84, total_fulfilled=74, total_backorders=10, fill_rate=0.8809523809523809, average_on_hand=2.3333333333333335, holding_cost=8.4, stockout_cost=40.0, total_cost=48.4)}


In [5]:
final_results

{'article-3001': SimulationResult(snapshots=[InventorySnapshot(period=0, starting_on_hand=30, demand=18, received=0, ending_on_hand=12, backorders=0, order_placed=19, on_order=19), InventorySnapshot(period=1, starting_on_hand=31, demand=20, received=19, ending_on_hand=11, backorders=0, order_placed=18, on_order=18), InventorySnapshot(period=2, starting_on_hand=29, demand=19, received=18, ending_on_hand=10, backorders=0, order_placed=19, on_order=19), InventorySnapshot(period=3, starting_on_hand=29, demand=21, received=19, ending_on_hand=8, backorders=0, order_placed=20, on_order=20), InventorySnapshot(period=4, starting_on_hand=28, demand=22, received=20, ending_on_hand=6, backorders=0, order_placed=20, on_order=20), InventorySnapshot(period=5, starting_on_hand=26, demand=20, received=20, ending_on_hand=6, backorders=0, order_placed=20, on_order=20)], summary=SimulationSummary(total_demand=120, total_fulfilled=120, total_backorders=0, fill_rate=1.0, average_on_hand=8.833333333333334, h

In [6]:
optimized_targets

{'article-3001': ForecastTargetOptimizationResult(target='45-low', policy=ForecastSeriesPolicy(forecast=[18, 19, 18, 19, 20, 20], lead_time=1), simulation=SimulationResult(snapshots=[InventorySnapshot(period=0, starting_on_hand=30, demand=18, received=0, ending_on_hand=12, backorders=0, order_placed=19, on_order=19), InventorySnapshot(period=1, starting_on_hand=31, demand=20, received=19, ending_on_hand=11, backorders=0, order_placed=18, on_order=18), InventorySnapshot(period=2, starting_on_hand=29, demand=19, received=18, ending_on_hand=10, backorders=0, order_placed=19, on_order=19), InventorySnapshot(period=3, starting_on_hand=29, demand=21, received=19, ending_on_hand=8, backorders=0, order_placed=20, on_order=20), InventorySnapshot(period=4, starting_on_hand=28, demand=22, received=20, ending_on_hand=6, backorders=0, order_placed=20, on_order=20), InventorySnapshot(period=5, starting_on_hand=26, demand=20, received=20, ending_on_hand=6, backorders=0, order_placed=20, on_order=20)]

In [7]:
results["article-1001"].snapshots[:3]


[InventorySnapshot(period=0, starting_on_hand=40, demand=15, received=0, ending_on_hand=25, backorders=0, order_placed=0, on_order=0),
 InventorySnapshot(period=1, starting_on_hand=25, demand=10, received=0, ending_on_hand=15, backorders=0, order_placed=50, on_order=50),
 InventorySnapshot(period=2, starting_on_hand=15, demand=25, received=0, ending_on_hand=0, backorders=10, order_placed=0, on_order=50)]